# **多类分类和神经网络——手写数字识别**

# One-vs-All Logistic Regression

One-vs-All (OvA), also called One-vs-Rest (OvR), is a strategy for extending binary classification algorithms like logistic regression to multi-class classification problems.

## How It Works

1. **For each class**, a binary classifier is trained to distinguish that class from all other classes combined
2. This creates **K separate binary classifiers** for a K-class problem
3. During prediction:
   - Each classifier outputs a probability score for its class
   - The class with the highest probability is selected as the final prediction

## Mathematical Formulation

For each class k (out of K total classes), we train a logistic regression model:

P(y = k | x) = σ(wₖᵀx + bₖ)

Where:
- σ is the sigmoid function: σ(z) = 1/(1 + e⁻ᶻ)
- wₖ is the weight vector for class k
- bₖ is the bias term for class k


## Advantages

- Simple to implement and understand
- Works well when the number of classes is not too large
- Can use any binary classification algorithm as the base classifier

## Disadvantages

- Can be computationally expensive for many classes (requires training K models)
- May suffer from class imbalance when one class is much smaller than others
- Ignores potential relationships between classes

## When to Use

- When your problem has a moderate number of classes (typically < 10)
- When base classifiers perform well on binary tasks
- When you need a simple, interpretable multi-class solution

Many machine learning libraries (like scikit-learn) implement this strategy automatically when you use their logistic regression implementation for multi-class problems.

In [204]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

data = loadmat('ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [205]:
def sigmoid(z:np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-z))

**正则化代价函数**：
$$J\left(\vec{w},b\right)=-\frac{1}{m}\sum_{i=1}^{m}\left[y^{\left(i\right)}\log{\left(f_{\vec{w},b}\left(\vec{x^{\left(i\right)}}\right)\right)}+\left(1-y^{\left(i\right)}\right)\log{\left(1-f_{\vec{w},b}\left(\vec{x^{\left(i\right)}}\right)\right)}\right]+\frac{\lambda}{2m}\sum_{j=1}^{n}w_j^2$$

In [206]:
def Regularized_CostFunction(theta: np.ndarray, X: np.ndarray, outcome: np.ndarray, Lambda: float) -> float:

    theta = np.array(theta).reshape(-1, 1)
    X = np.array(X)
    outcome = np.array(outcome).reshape(-1, 1)
    
    m = outcome.shape[0]  # Number of training examples
    z = X @ theta
    h = sigmoid(z)

    cost = - np.sum(outcome.T @ np.log(h) + (1 - outcome).T @ np.log(1 - h)) / m
    reg = (Lambda / (2 * m)) * np.sum(theta[1:]**2)  # Skip theta[0] for regularization

    return float(cost + reg)

In [207]:
# 训练模型时候转换y为0-1
def RegularizedCostFunction(theta:np.ndarray, X:np.ndarray, y:np.ndarray, lambda_:float) -> float:
    m = len(y)
    theta = theta.reshape(-1, 1)
    y = y.reshape(-1, 1)

    h = sigmoid(X @ theta) # m*n * n*1 = m*1
    cost = (-1/m) * np.sum(y.T @ (np.log(h)) +  (1 - y).T @ (np.log(1 - h))  )# np.sum() is used to extract element in the 1*1 array
    reg_cost = (lambda_ / (2 * m)) * np.sum(np.square(theta[1:]))
    return cost + reg_cost

X = data['X']
# Add intercept term
X = np.hstack((np.ones((X.shape[0], 1)), X)) # hstack水平叠加，np.ones创建全一矩阵
# X_with_bias = np.insert(X, 0, 1, axis=1)  # 在第0列插入全1

y = data['y']
theta = np.zeros(X.shape[1])
lambda_ = 1.0
cost = RegularizedCostFunction(theta, X, y, lambda_)
print(f"Regularized Cost: {cost}")

Regularized Cost: 160.39425758157174


$$\theta_0:=\theta_0-\alpha\frac{1}{m}\sum_{i=1}^{m}{\left[f_{\vec{w},b}\left({\vec{x}}^{\left(i\right)}\right)-y^{\left(i\right)}\right]x_0^{\left(i\right)}}$$
$$\theta_j:=\theta_j-\alpha\frac{1}{m}\sum_{i=1}^{m}{f_{\vec{w},b}\left[\left({\vec{x}}^{\left(i\right)}\right)-y^{\left(i\right)}\right]x_j^{\left(i\right)}}+\alpha\frac{\lambda}{m}\theta_j$$

In [208]:
def RegularizedGradientDescent_auto(theta:np.ndarray, X:np.ndarray, y:np.ndarray, initial_alpha:float, lambda_:float, max_iter:int, tolerance: float) -> list:
    m = len(y)
    theta = theta.reshape(-1, 1)
    y = y.reshape(-1, 1)

    pre_cost = float('inf')
    for i in range(max_iter):
        error = sigmoid(X @ theta) - y # m*n * n*1 = m*1
        gradient = (1/m) * (X * error)
        gradient = np.sum(gradient, axis=0).reshape(-1, 1)
        theta[0] -= initial_alpha * gradient[0] 
        for i in range(1, len(theta)):
            theta[i] -= initial_alpha * (gradient[i] - (lambda_ / m) * theta[i])
        cur_cost=(RegularizedCostFunction(theta, X, y, lambda_))
        if abs(pre_cost - cur_cost) < tolerance :
            break
        if pre_cost < cur_cost:
            initial_alpha *= 0.5
        else:
            initial_alpha *= 1.1
        pre_cost = cur_cost

In [209]:
y

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=uint8)

In [210]:
def OneVsAll(X:np.ndarray, y:np.ndarray, num_labels:int, initial_alpha:float, lambda_:float, max_iter:int, tolerance: float) -> np.ndarray:
    m, n = X.shape
    all_theta = np.zeros((num_labels, n))
    
    for c in range(num_labels):
        # binary_y = y == (c+1)
        binary_y = np.where(y == (c + 1) , 1, 0)
        initial_theta = np.zeros(n)
        RegularizedGradientDescent_auto(initial_theta, X, binary_y, initial_alpha, lambda_, max_iter, tolerance)
        all_theta[c] = initial_theta

    return all_theta

In [211]:
all_theta = OneVsAll(X, y, 10, 1, 0.1, 10000, 1e-20)

In [212]:
all_theta

array([[-3.10098247e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.30906348e-03,  3.13494912e-07,  0.00000000e+00],
       [-3.84094003e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.27338522e-02, -1.45420572e-03,  0.00000000e+00],
       [-5.93540881e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.11047350e-05, -2.50675186e-07,  0.00000000e+00],
       ...,
       [-9.41759719e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -4.06482449e-04,  3.90034602e-05,  0.00000000e+00],
       [-5.73952453e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -6.68533036e-03,  5.29220024e-04,  0.00000000e+00],
       [-6.87515205e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -4.19337995e-04,  1.97398919e-05,  0.00000000e+00]])

In [213]:
def predict_all(X, all_theta):
    
    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T) # m*n * n*k = m*k
    
    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)
    print(h_argmax)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1
    
    return h_argmax

In [214]:
y_pred = predict_all(X, all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

[[9]
 [9]
 [9]
 ...
 [8]
 [8]
 [8]]
accuracy = 96.24000000000001%


In [218]:
all_theta2 = OneVsAll(X, y, 10, 0.1, 1, 10000, 1e-8)
all_theta2

array([[-1.41143500e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.93055358e-04,  1.79122523e-07,  0.00000000e+00],
       [-2.64020469e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.36424484e-03, -2.71934725e-04,  0.00000000e+00],
       [-4.09366113e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.43230235e-05,  1.66804888e-08,  0.00000000e+00],
       ...,
       [-7.28614575e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.23158261e-04,  1.13727980e-05,  0.00000000e+00],
       [-4.09295473e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -4.01060495e-04,  2.42297992e-05,  0.00000000e+00],
       [-3.79469762e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -3.59416326e-04,  1.09752741e-05,  0.00000000e+00]])

In [216]:
y_pred = predict_all(X, all_theta2)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

[[9]
 [9]
 [9]
 ...
 [8]
 [8]
 [6]]
accuracy = 93.96%
